Part 1: building a sample Check the status

notebook to have data generation for recommandation engine 

Installation of awsrangler library you need to have version 15 

In [ ]:
pip install awswrangler==2.15.0

configure your connection and environement 

In [1]:
import awswrangler as wr
import pandas as pd

port = 8182
url='yourcluster endpoint' # The Neptune Cluster endpoint

iam_enabled = False # Set to True/False based on the configuration of your cluster
neptune_port = 8182 # Set to the Neptune Cluster Port, Default is 8182
client = wr.neptune.connect(url, neptune_port, iam_enabled=iam_enabled)

check if this is empty 

In [5]:

query = "g.V().has('id', 100 ).out('know').valueMap().path()"

df = wr.neptune.execute_gremlin(client, query)
display(df.head(5))

,0,1,2
0,"{'id': '100', 'label': 'person'}","{'id': '403', 'label': 'person'}","{'val': 'n', 'name': 'nhhaktstfi', 'id': 403, ..."
1,"{'id': '100', 'label': 'person'}","{'id': '609', 'label': 'person'}","{'val': 'y', 'name': 'ypvpetymce', 'id': 609, ..."
2,"{'id': '100', 'label': 'person'}","{'id': '638', 'label': 'person'}","{'val': 'y', 'name': 'wxjqfmwvnt', 'id': 638, ..."
3,"{'id': '100', 'label': 'person'}","{'id': '277', 'label': 'person'}","{'val': 'y', 'name': 'wbjkvfitxh', 'id': 277, ..."
4,"{'id': '100', 'label': 'person'}","{'id': '287', 'label': 'person'}","{'val': 'y', 'name': 'qygxqxrzkd', 'id': 287, ..."


In [ ]:
if you want to delete the graph use the following command 

In [6]:
%%gremlin
g.V().drop().iterate()

""


In [ ]:
import random
import string


def _create_person_vertex(i):
    list1 = ['y', 'n']
    data = dict()
    data["~id"] = i
    data["~label"] = "person"
    data["id"] = i
    data["name"] = "".join(random.choice(string.ascii_lowercase) for i in range(10))
    data["val"] = random.choice(list1)
    data["sample"] = [random.randint(0, 1000), random.randint(0, 1000)]
    return data


r1 = range(100,1000)
for i in r1 : 
    data = [_create_person_vertex(i)]
    df = pd.DataFrame(data)
    res = wr.neptune.to_property_graph(client, df)
    query = f"MATCH (s) WHERE id(s)='{data[0]['id']}' RETURN s"
    df = wr.neptune.execute_opencypher(client, query)
    display(df)


In [ ]:
import uuid
import random
import string


def _create_object_vertex(i):
    list1 = ['email', 'phone']
    data = dict()
    data["~id"] = i
    data["~label"] = "object"
    data["id"] = i
    data["name"] = "".join(random.choice(string.ascii_lowercase) for i in range(10))
    data["val"] = random.choice(list1)
    return data

r1 = range(1,99)
for i in r1 : 
    data = [_create_object_vertex(random.randint(1, 99))]
    df = pd.DataFrame(data)
    res = wr.neptune.to_property_graph(client, df)

In [ ]:
import uuid
import random
import string

def _create_person_edge(i,j):
    list1 = ['y', 'n']
    data = dict()
    data["~id"] = uuid.uuid4()
    data["~label"] = "know"
    data["~to"] = i
    data["~from"] = j
    data["int"] = random.randint(0, 1000)
    data["par"] = random.choice(list1)
    data["str"] = "".join(random.choice(string.ascii_lowercase) for i in range(10))
    return data

r1 = range(1,5000)
for i in r1 : 
    data = [_create_person_edge(random.randint(100, 1000),random.randint(100, 1000))]
    df = pd.DataFrame(data)
    res = wr.neptune.to_property_graph(client, df)


In [17]:
import uuid
import random
import string

def _create_object_edge(i,j):
    data = dict()
    data["~id"] = uuid.uuid4()
    data["~label"] = "belong"
    data["~to"] = i
    data["~from"] = j
    data["int"] = random.randint(0, 1000)
    return data

r1 = range(1,1000)
for i in r1 : 
    data = [_create_object_edge(random.randint(1, 99),random.randint(100, 1000))]
    df = pd.DataFrame(data)
    res = wr.neptune.to_property_graph(client, df)

check the status to connect to graph db 

In [32]:
%status

{'status': 'healthy',
 'startTime': 'Thu Apr 14 06:08:41 UTC 2022',
 'dbEngineVersion': '',
 'role': 'writer',
 'gremlin': {'version': 'tinkerpop-3.4.11'},
 'sparql': {'version': 'sparql-1.1'},
 'labMode': {'ObjectIndex': 'disabled',
  'DFEQueryEngine': 'viaQueryHint',
  'ReadWriteConflictDetection': 'enabled'},
 'features': {'ResultCache': {'status': 'disabled'},
  'IAMAuthentication': 'disabled',
  'Streams': 'enabled',
  'AuditLog': 'enabled'},
 'settings': {'clusterQueryTimeoutInMs': '300000'}}

get status and drop Exisitng vertices and edge to clean your graph 

In [ ]:
%%gremlin
g.V().drop().iterate()

View all the graph to show the global view 

In [30]:
%%gremlin

g.V().both().valueMap().path()

count the number of edges and vertex 

In [33]:
%%gremlin


g.V().count()


In [34]:
%%gremlin


g.E().count()

show a sample for a specific ID to illustrate the connection 

In [35]:
%%gremlin

g.V().                   // Iterator<Vertex>
  has( "id", 400 ). // Iterator<User>
  bothE(). 
  // 2nd degree friends
  simplePath().
  dedup().
  valueMap().path() 

In [ ]:
View all the activities play by one user here a par property autogenerate

In [ ]:
na = 'Exx'

In [ ]:
%%gremlin


g.V().has('str, '${na}').out('know').valueMap().path()

View all Bill Friends 

In [ ]:

%%gremlin

g.V().has('name', '${na}').out('know').valueMap().path()



let's now create a PYMK recommendation for a specific user.

In the query below, we're finding the vertex that represents our user. We're then traversing FRIEND relationships (we don't care about relationship direction, so we're using both()) to find that user's immediate friends. We're then traversing another hop into the graph, looking for friends of those friends who are not currently connected to our user (i.e., we're looking for the unclosed triangles).

We then count the paths to these candidate friends, and order the results based on the number of times we can reach a candidate via one of the user's immediate friends.

In [37]:
%%gremlin

g.V().hasLabel('person').has('id', 400).as('person').  
  both('know').aggregate('know').  
  both('know').
    where(P.neq('person')).where(P.without('know')).  
  groupCount().by('name').  
  order(Scope.local).by(values, Order.decr).
  next()

In [38]:
%%gremlin

g.V().                   // Iterator<Vertex>
  has("id", 400 ). // Iterator<User>
  out("know"). 
  out("know"). 
  where(not(has("name", "Emmanuel"))).
 // 2nd degree friends
  dedup().
  valueMap().path()   


In [40]:
%%gremlin

g.V().                   // Iterator<Vertex>
  has("id", 400 ). 
  outE("know").has('par',"y").inV().
   where(not(has("id", 400))).
 // 2nd degree friends
  dedup().
  valueMap().path()   


In [41]:
%%gremlin

g.V().                   // Iterator<Vertex>
  has("id", 400 ). 
  outE("know").has('par',"y").inV().
  outE("know").has('par',"y").inV().
  where(not(has("name", "Emmanuel"))).
 // 2nd degree friends
  dedup().
  valueMap().path()   



In [28]:
%%gremlin

g.V().property(single, 'Importance',both().count())

Same for other basket or bike 

In [29]:
%%gremlin 

g.V().valueMap()

let's now find a object and see all the relation of this obhect 

In [21]:
%%gremlin


g.V().has('id', 20).in('belong').valueMap().path()

In [42]:
%%gremlin

g.V().                   // Iterator<Vertex>
  has("name", "wzyssuvjdj" ). // Iterator<User>
  outE("belong"). 
  inV().in('belong').dedup().simplePath().valueMap().path()


In [44]:
%%oc
MATCH (n)
RETURN n
LIMIT 5

Tab()

who the person with ID 400 know 

In [48]:
%%oc -d $my_node_labels -de $my_edge_labels -l 20
MATCH p = (a:person {id:400})-[:know]->()
RETURN p

Tab()